In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#-------------------------------------------------
# 1) LOAD DATA
#-------------------------------------------------
# Example: Suppose your data is in a CSV with columns:
#   time, pressure, range_CE, range_EC
# Replace 'your_data.csv' with the actual file name.
data_file = 'your_data.csv'

df = pd.read_csv(data_file)

# Extract columns; make sure these match your actual column names
time   = df['time'].values            # e.g. in days
p      = df['pressure'].values        # differential pressure time series
r_CE   = df['range_CE'].values        # distance from station C to E
r_EC   = df['range_EC'].values        # distance from station E to C

#-------------------------------------------------
# 2) DEFINE PERIOD & BUILD SIN/COS
#-------------------------------------------------
# For an annual signal, typical period is ~365 days.
# Adjust if you want a more precise or different period.
T_year = 365.25
omega  = 2.0 * np.pi / T_year

sin_term = np.sin(omega * time)
cos_term = np.cos(omega * time)

#-------------------------------------------------
# 3) BUILD DESIGN MATRIX
#-------------------------------------------------
# Model: R(t) = a * P(t) + b * sin(2πt/T) + c * cos(2πt/T)
# We'll create the same design matrix for both time series
# because the physical model is similar; we just solve them separately.
A = np.column_stack([p, sin_term, cos_term])
# A has shape (N, 3), where N is the number of time samples.

#-------------------------------------------------
# 4) LEAST SQUARES FOR R_CE
#-------------------------------------------------
# Solve for x = (a, b, c) for station C→E
x_CE, residuals_CE, rank_CE, s_CE = np.linalg.lstsq(A, r_CE, rcond=None)
a_CE, b_CE, c_CE = x_CE

# Create the modeled range for station C→E
r_CE_fit = A.dot(x_CE)

#-------------------------------------------------
# 5) LEAST SQUARES FOR R_EC
#-------------------------------------------------
# Solve for x = (a, b, c) for station E→C
x_EC, residuals_EC, rank_EC, s_EC = np.linalg.lstsq(A, r_EC, rcond=None)
a_EC, b_EC, c_EC = x_EC

# Create the modeled range for station E→C
r_EC_fit = A.dot(x_EC)

#-------------------------------------------------
# 6) PRINT RESULTS
#-------------------------------------------------
print("==== Model Coefficients for Range C→E ====")
print(f"a_CE (Pressure coeff): {a_CE:.6f}")
print(f"b_CE (Annual sine):    {b_CE:.6f}")
print(f"c_CE (Annual cosine):  {c_CE:.6f}")

print("\n==== Model Coefficients for Range E→C ====")
print(f"a_EC (Pressure coeff): {a_EC:.6f}")
print(f"b_EC (Annual sine):    {b_EC:.6f}")
print(f"c_EC (Annual cosine):  {c_EC:.6f}")

#-------------------------------------------------
# 7) OPTIONAL: PLOT RESULTS
#-------------------------------------------------
plt.figure()
plt.title("Station C→E: Observed vs. Modeled Range")
plt.plot(time, r_CE, 'o', label="Observed R_CE", markersize=3)
plt.plot(time, r_CE_fit, '-', label="Modeled R_CE Fit")
plt.xlabel("Time (days)")
plt.ylabel("Range (C→E)")
plt.legend()
plt.show()

plt.figure()
plt.title("Station E→C: Observed vs. Modeled Range")
plt.plot(time, r_EC, 'o', label="Observed R_EC", markersize=3)
plt.plot(time, r_EC_fit, '-', label="Modeled R_EC Fit")
plt.xlabel("Time (days)")
plt.ylabel("Range (E→C)")
plt.legend()
plt.show()
